In [2]:
! pip install pyspark
! pip install py4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.2/317.2 MB 56.0 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyspark: filename=pyspark-3.5.5-py2.py3-none-any.whl size=317747920 sha256=a72a54ae0a049a3eab7abcb99356f394b0493fb3d86383f7c400749e58711cd9
  Stored in directory: /home/codespace/.cache/pip/wheels/8f/cb/c0/cc57eb1bf0f9dc87cdaf2b0dbac49e58a210ff68d21d6fc709
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("CSVtoParquet") \
    .getOrCreate()

25/04/13 15:59:23 WARN Utils: Your hostname, codespaces-240bfe resolves to a loopback address: 127.0.0.1; using 10.0.12.140 instead (on interface eth0)
25/04/13 15:59:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/13 15:59:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
tables = [
    "categories",
    "cities",
    "countries",
    "customers",
    "employees",
    "products",
    "sales"
    ]

In [6]:
def create_dataframe(file_name):
    df = spark.read \
    .option("header", "true") \
    .option("inferSchema",True) \
    .csv(file_name)
    return df


In [ ]:
schemas = {}
for table in tables:
    df = create_dataframe(f"../data/raw/{table}.csv")
    schemas[table] = df.schema 

In [14]:
schemas

{'categories': StructType([StructField('CategoryID', IntegerType(), True), StructField('CategoryName', StringType(), True)]),
 'cities': StructType([StructField('CityID', IntegerType(), True), StructField('CityName', StringType(), True), StructField('Zipcode', IntegerType(), True), StructField('CountryID', IntegerType(), True)]),
 'countries': StructType([StructField('CountryID', IntegerType(), True), StructField('CountryName', StringType(), True), StructField('CountryCode', StringType(), True)]),
 'customers': StructType([StructField('CustomerID', IntegerType(), True), StructField('FirstName', StringType(), True), StructField('MiddleInitial', StringType(), True), StructField('LastName', StringType(), True), StructField('CityID', IntegerType(), True), StructField('Address', StringType(), True)]),
 'employees': StructType([StructField('EmployeeID', IntegerType(), True), StructField('FirstName', StringType(), True), StructField('MiddleInitial', StringType(), True), StructField('LastName'

In [ ]:
{
'categories': 
StructType([
    StructField('CategoryID', IntegerType(), True), 
    StructField('CategoryName', StringType(), True)
    ]),

'cities': 
StructType([
    StructField('CityID', IntegerType(), True), 
    StructField('CityName', StringType(), True), 
    StructField('Zipcode', IntegerType(), True), 
    StructField('CountryID', IntegerType(), True)
    ]),

'countries': 
StructType([
    StructField('CountryID', IntegerType(), True), 
    StructField('CountryName', StringType(), True), 
    StructField('CountryCode', StringType(), True)
    ]),

'customers': 
StructType([
    StructField('CustomerID', IntegerType(), True),
    StructField('FirstName', StringType(), True), 
    StructField('MiddleInitial', StringType(), True), 
    StructField('LastName', StringType(), True), 
    StructField('CityID', IntegerType(), True), 
    StructField('Address', StringType(), True)
    ]),

'employees': 
StructType([
    StructField('EmployeeID', IntegerType(), True), 
    StructField('FirstName', StringType(), True), 
    StructField('MiddleInitial', StringType(), True), 
    StructField('LastName', StringType(), True), 
    StructField('BirthDate', TimestampType(), True), 
    StructField('Gender', StringType(), True), 
    StructField('CityID', IntegerType(), True), 
    StructField('HireDate', TimestampType(), True)
    ]),

'products': 
StructType([
    StructField('ProductID', IntegerType(), True), 
    StructField('ProductName', StringType(), True), 
    StructField('Price', DoubleType(), True), 
    StructField('CategoryID', IntegerType(), True),
    StructField('Class', StringType(), True), 
    StructField('ModifyDate', TimestampType(), True), 
    StructField('Resistant', StringType(), True), 
    StructField('IsAllergic', StringType(), True), 
    StructField('VitalityDays', DoubleType(), True)
    ]),

'sales': 
StructType([
    StructField('SalesID', IntegerType(), True),
    StructField('SalesPersonID', IntegerType(), True), 
    StructField('CustomerID', IntegerType(), True), 
    StructField('ProductID', IntegerType(), True), 
    StructField('Quantity', IntegerType(), True), 
    StructField('Discount', DoubleType(), True), 
    StructField('TotalPrice', DoubleType(), True), 
    StructField('SalesDate', TimestampType(), True),
    StructField('TransactionNumber', StringType(), True)
      ])}

In [15]:
import os

def create_directory(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

In [ ]:

for table in tables:
    df = create_dataframe(f"../data/raw/{table}.csv")
    directory = f"../data/datalake/{table}"
    create_directory(directory)
    df.write.mode("overwrite").parquet(f"../data/datalake/{table}/{table}.parquet")